# Tuberculosis Detection

In [1]:
from utils import load_dataframe
import fastai
from fastai.vision.all import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

/home/costadiego/libs/anaconda3/envs/tuberculosis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generate the dataframe

In [2]:
dataset_description_path = "dataset_description.yaml"

In [3]:
#Generate dataset from description
tb_df = load_dataframe(dataset_description_path)

In [4]:
tb_df

,file,path,abnormal,tuberculosis
0,00000562_000.png,dataset/anormal-xr-no_tb/00000562_000.png,1,0
1,00000199_002.png,dataset/anormal-xr-no_tb/00000199_002.png,1,0
2,00000628_000.png,dataset/anormal-xr-no_tb/00000628_000.png,1,0
3,00000897_000.png,dataset/anormal-xr-no_tb/00000897_000.png,1,0
4,00000830_006.png,dataset/anormal-xr-no_tb/00000830_006.png,1,0
...,...,...,...,...
15753,Tuberculosis-49.png,dataset/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-49.png,1,1
15754,Tuberculosis-127.png,dataset/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-127.png,1,1
15755,Tuberculosis-166.png,dataset/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-166.png,1,1
15756,Tuberculosis-237.png,dataset/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-237.png,1,1


In [5]:
train_test_df, val_df = train_test_split(tb_df, test_size=0.1)

In [6]:
print("Train/Test Dataframe:", train_test_df.shape)
print("Validation Dataframe:", val_df.shape)

Train/Test Dataframe: (14182, 4)
Validation Dataframe: (1576, 4)


## Configuring the Dataloader

In [7]:
def train_test_splitter(dataframe):
    train_df, test_df = train_test_split(dataframe, test_size=0.2)
    train_df.index.tolist()
    test_df.index.tolist()
    return (train_df, test_df)
def get_image_input(row):
    return row['path']
def get_labels(row):
    return row['abnormal'],row['tuberculosis']
def get_abnormal(row):
    return row['abnormal']
def get_tuberculosis(row):
    return row['tuberculosis']

In [8]:
# tfms = [Normalize.from_stats([0.0692], [0.2051]),  # Normalize the images with the specified mean and standard deviation
#         *aug_transforms(do_flip=False, size=128)]  # Add default transformations except for horizontal flip    
# dblock = DataBlock(
#   blocks=(ImageBlock(cls=PILImageBW), *(2*[CategoryBlock])),      # one image input and three categorical targets
#   getters=[ColReader('path'),   # image input
#            ColReader('abnormal'),                            # label 1
#            ColReader('tuberculosis')],                       # label 2
#   splitter=train_test_splitter,                 # train/validation split
#   batch_tfms=tfms)                                           # Add normalization and augmentations  
# dls = dblock.dataloaders(train_test_df, bs=64)   # Create the dataloaders
# dls.n_inp = 1  # Set the number of inputs

In [9]:
tfms = [Normalize.from_stats([0.0692], [0.2051]),  # Normalize the images with the specified mean and standard deviation
        *aug_transforms(do_flip=False, size=128)]  # Add default transformations except for horizontal flip    
dblock = DataBlock(
  blocks=(ImageBlock(cls=PILImageBW), *(2*[CategoryBlock])),      # one image input and three categorical targets
  getters=[ColReader(1),   # image input
           ColReader(2),                            # label 1
           ColReader(3)],                       # label 2
  splitter=train_test_splitter,                 # train/validation split
  batch_tfms=tfms)                                           # Add normalization and augmentations  
dls = dblock.dataloaders(train_test_df, bs=5)   # Create the dataloaders
dls.n_inp = 1  # Set the number of inputs

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
dls.show_batch(nrows=1, ncols=3)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.5))